In [10]:
import sys, os
import numpy as np
import cv2
import scipy.io as sio
import tqdm

In [11]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
from sklearn.neighbors import NearestNeighbors

In [12]:
def vid2img(filename):
    cap=cv2.VideoCapture(filename)
    success,frame=cap.read()
    allImg=[]
    while(success):
        allImg.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        success,frame=cap.read()
    return allImg

def allImg2vid(allImg, output_vid='output_body.mp4', frameRate=30):
    vidCodec = cv2.VideoWriter_fourcc(*'XVID')
    height, width, channel = allImg[0].shape
    vidFile = cv2.VideoWriter(output_vid, vidCodec, frameRate, (width, height))
    for i in tqdm.tqdm(range(0,len(allImg))):
        frame = cv2.cvtColor(allImg[i],cv2.COLOR_RGB2BGR)
        vidFile.write(frame)
    vidFile.release()
    print(" Successfully converted images in allImg to "+output_vid)
    return None


In [13]:
def get_dynamic_video(pred_lm_fps, input_pred_lm_filename, target_lm_filename, input_video_filename, target_video_filename):
#     input_pred_lm_filename is the preddicted 20D landmarks .mat  file format with y_pred as the key for landmarks
#     target_lm_filename is the target video 68D landmarks in .mat file format with ypred as the key for the landmarks
#     default pred landmark fps = 25
#     input_video_filename is the video corresponding to input_pred_lm_filename
#     target_video_filename is the target video filename
    print('reading file:'+input_pred_lm_filename)
    lm_src=sio.loadmat(input_pred_lm_filename)['y_pred']
    print('reading file:'+target_lm_filename)
    full_lm_dst=sio.loadmat(target_lm_filename)['ypred']
    lm_dst=full_lm_dst[:,-20:]
    lm_center_src = lm_src[:,[14,16]]
    lower_lm_src = lm_center_src[:,1,0]
    lower_lm_src = (lower_lm_src-lower_lm_src.min())/(lower_lm_src.max()-lower_lm_src.min())
    
    lm_center_dst = lm_dst[:,[14,16]]
    lower_lm_dst = lm_center_dst[:,1,0]
    lower_lm_dst = (lower_lm_dst-lower_lm_dst.min())/(lower_lm_dst.max()-lower_lm_dst.min())
    
    k_centers=np.array([0.0,0.2,0.4,0.6,0.8,1.0]).astype('float')
    
    
    neigh = NearestNeighbors(1)
    neigh.fit(np.expand_dims(lower_lm_dst,1))
    dst_centroids=neigh.kneighbors(np.expand_dims(k_centers,1), 1, return_distance=False)
    
    neigh = NearestNeighbors(1)
    neigh.fit(np.expand_dims(k_centers,1))
    src_assignment=neigh.kneighbors(np.expand_dims(lower_lm_src,1), 1, return_distance=False)
    
    dst_frame_ids_for_source=dst_centroids[src_assignment].squeeze()
    temp=[]
    for i in range(0,5):
        temp.append(dst_frame_ids_for_source[0])
    dst_frame_ids_for_source=np.array(temp+dst_frame_ids_for_source.tolist())
    
    print('reading file:'+target_video_filename)
    allImg_im1=vid2img(target_video_filename)
    allImg_dst2src=[]
    dynamic_lm_dst=[]
    for i in dst_frame_ids_for_source:
        allImg_dst2src.append(allImg_im1[i])
        dynamic_lm_dst.append(full_lm_dst[i])
    dynamic_lm_dst=np.array(dynamic_lm_dst)
    dynamic_lm_dst[0]
    sio.savemat('Dynamic_LM.mat',{'lm':dynamic_lm_dst})
    allImg2vid(allImg_dst2src,'output_with_jaw.mp4',frameRate=pred_lm_fps)
    print('Running shell script')
    shell_command='ffmpeg -i output_with_jaw.mp4 -i '+input_video_filename+' -c copy -map 0:v:0 -map 1:a:0 -shortest out_with_jaw_body.mp4'
    os.system(shell_command)
    return None

In [15]:
get_dynamic_video(pred_lm_fps=25, input_pred_lm_filename='CV_01_C4W1L01_000003_to_000045_hindi_abhishek_generated_lip_landmarks.mat', target_lm_filename='ANDREW_NG_CV_01_C4W1L01_000003_to_000045_landmarks_in_frames.mat', input_video_filename='input_videos/CV_01_C4W1L01_000003_to_000045/CV_01_C4W1L01_000003_to_000045_hindi_abhishek_making.mp4', target_video_filename='input_videos/CV_01_C4W1L01_000003_to_000045/CV_01_C4W1L01_000003_to_000045.mp4')

reading file:CV_01_C4W1L01_000003_to_000045_hindi_abhishek_generated_lip_landmarks.mat
reading file:ANDREW_NG_CV_01_C4W1L01_000003_to_000045_landmarks_in_frames.mat
reading file:input_videos/CV_01_C4W1L01_000003_to_000045/CV_01_C4W1L01_000003_to_000045.mp4


100%|██████████| 1107/1107 [00:05<00:00, 185.48it/s]


 Successfully converted images in allImg to output_with_jaw.mp4
Running shell script
